# Rete neurale per extreme returns su 2 azioni

Questo notebook contiene la parte di rete neurale per confronto con l'analisi statistica.

Il flusso è il seguente:

- [ ] utilizzo del dataset *S&P500* con la massima ampiezza storica disponibile (2005 - 2018)
- [ ] calcolo dei log returns
- [ ] selezione di due stocks, quelle con la minima e la massima volatilità in nel training set considerato
- [ ] creazione estremi al 95%
- [ ] oversampling aggiungendo del rumore gaussiano
- [ ] addestramento rete con hyperparameter optimization
- [ ] utilizzo stesse metriche (ROC, KSS, Precision, Recall, Utility) che nel paper
- [ ] confronto con i risultati del modello probabilistico
- [ ] conclusioni

In [ ]:
import os
import time
import datetime
from typing import List, Union
import itertools
import pickle
import math
import copy

import numpy as np
import pandas as pd
import pandas.testing as pt
import scipy.stats
import scipy.special as sfun
from scipy.stats import genextreme as gev
import sklearn.metrics as sm

from statsmodels.tsa import stattools
from statsmodels.graphics import tsaplots

import matplotlib.pyplot as pl
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import seaborn as sns

from tqdm import tqdm
import ipdb

import numba

# import del @timeit
from my_timeit import timeit

%pdb on
%load_ext line_profiler
%load_ext autoreload

%autoreload 5

Un po' di dichiarazioni utili per il seguito

In [ ]:
stock_type = ['min_vol', 'max_vol']
return_type = ['pos', 'neg', 'abs']
q_type = '95'

stock_codes = {
    'min_vol': '9CE4C7',
    'max_vol': 'E28F22'
}  # già trovate in Paper-azioni.ipynb

stock_colors = {
    'min_vol': 'palegoldenrod',
    'max_vol': 'coral',
}

# i giorni sono i primi disponibili in quel mese nei dati
split_dates = {
    'subprime-crisis': datetime.datetime(2007, 1, 3), # subprime crisis
    'subprime-crisis-start': datetime.datetime(2007, 1, 3), # subprime crisis
    'subprime-crisis-halfway': datetime.datetime(2008, 9, 2),
    'subprime-crisis-end': datetime.datetime(2010, 1, 4),
    'eu-debt': datetime.datetime(2011, 1, 3), # EU sovereign debt crisis
    'eu-debt-halfway': datetime.datetime(2012, 1, 3), # EU sovereign debt crisis
    'last_train': datetime.datetime(2017, 1, 3), 
}

## 1. Importazione dei dati 

Per importare i dati dobbiamo caricarli, e poi usare la stategia "taglia-e-cuci" usata in `Paper-azioni.ipynb`.

In [ ]:
data_path = "/Users/pietro/Google Drive/OptiRisk Thesis/data"
prices_path = os.path.join(data_path, 'prices', 'adjusted_prices_volume.csv')

Conversione delle date e settaggio dell'index del dataframe

In [ ]:
prices = pd.read_csv(prices_path)
prices.loc[:, 'date'] = pd.to_datetime(prices['date'], format="%Y%m%d")
prices.index = prices['date']
prices.drop(columns=['date'], inplace=True)
prices.head()

Trasformiamola un una serie temporale, ogni riga una data, ogni colonna un'azione.

I prezzi:

In [ ]:
prices_ts = prices.pivot(columns='ravenpackId', values='close')
prices_ts_no_nan = prices_ts.dropna(axis='columns', how='any', inplace=False)
prices_ts_no_nan.head()

I volumi:

In [ ]:
volume_ts = prices.pivot(columns='ravenpackId', values='volume')
volume_ts_no_nan = volume_ts.loc[:, prices_ts_no_nan.columns]
pt.assert_index_equal(prices_ts_no_nan.columns, volume_ts_no_nan.columns, check_names=False)
volume_ts_no_nan.head()

Ora calcoliamo i log-returns:

In [ ]:
log_returns = np.log(prices_ts_no_nan).diff(periods=1).iloc[1:, :]
prices_ts_no_nan = prices_ts_no_nan.iloc[1:, :]
volume_ts_no_nan = volume_ts_no_nan.iloc[1:, :]

pt.assert_index_equal(prices_ts_no_nan.index, volume_ts_no_nan.index)
pt.assert_index_equal(prices_ts_no_nan.index, log_returns.index)

In [ ]:
lr_temp = {
    s_type: log_returns.loc[:, stock_codes[s_type]]
    for s_type in stock_type
}

lr_train_for_percentiles = dict()
lr_test_for_percentiles = dict()
returns_train_for_percentiles = dict()

# aggiungiamo i dati in modalità taglia-e-cuci
for s_type in stock_type:
    # training set
    lr_train_1 = lr_temp[s_type] \
        [lr_temp[s_type].index < split_dates['subprime-crisis-halfway']]
    lr_train_2 = lr_temp[s_type] \
        [(lr_temp[s_type].index >= split_dates['eu-debt-halfway']) & \
         (lr_temp[s_type].index < split_dates['last_train'])]
    
    lr_train_for_percentiles[s_type] = pd.concat([lr_train_1, lr_train_2], axis='index')
    
    # test set
    lr_test_1 = lr_temp[s_type] \
        [lr_temp[s_type].index >= split_dates['last_train']]
    lr_test_2 = lr_temp[s_type] \
        [(lr_temp[s_type].index < split_dates['eu-debt-halfway']) & \
         (lr_temp[s_type].index >= split_dates['subprime-crisis-halfway'])]
    
    lr_test_for_percentiles[s_type] = pd.concat([lr_test_1, lr_test_2], axis='index')
    
    # returns train, tutti POSITIVI
    returns_train_for_percentiles[s_type] = {
        'pos': lr_train_for_percentiles[s_type][lr_train_for_percentiles[s_type] > 0.0],
        'neg': -(lr_train_for_percentiles[s_type][lr_train_for_percentiles[s_type] < 0.0]),
        'abs': lr_train_for_percentiles[s_type].abs()
    }

Bene, ora che ho i dataset così creati, uso il training set per calcolare i thresholds con i percentili al 95%:

In [ ]:
thresholds = {
    s_type: {
        ret_type: {
            q_type: returns_train_for_percentiles[s_type][ret_type].quantile(0.95)
        }
        for ret_type in return_type
    }
    for s_type in stock_type
}

## 2. Creazione dataset train-test

Ora che ho i thresholds, posso creare il dataset vero e proprio, cioè:

- X: cubo dati
- y: estremo si/no

Per prima cosa, creo le due serie di estremi/non estremi, che poi potrò spezzettare:

In [ ]:
extremes_all = dict()
returns_all = dict()

for s_type, s_code in stock_codes.items():
    returns_all[s_type] = log_returns.loc[:, s_code]
    extremes_all[s_type] = dict()
    
    ext = np.logical_or(
        log_returns.loc[:, s_code] >= thresholds[s_type]['pos'][q_type],
        log_returns.loc[:, s_code] <= -thresholds[s_type]['neg'][q_type],
    )
    
    extremes_all[s_type][q_type] = pd.Series(data=ext, index=log_returns.index)

### 2.1 Funzioni per la creazione dati

Ora, con i returns e gli estremi già calcolati, possiamo creare i dataset di training e testing.

L'idea è di dare i dati completi, l'indice di inizio, l'indice di fine e la bptt a una funzione che crea il cubo `X` e il vettore `y`, e restituisce anche le date `dates`.

Cominciamo con quella che ritorna il cubo `X`:

In [ ]:
# testata, funziona con array, Series e DataFrame
def rolling_window(data: Union[np.ndarray, pd.Series, pd.DataFrame],
                   start: int,
                   end: int,
                   lookback: int):
    """
    Create a rolling window view of data, starting at index start, finishing
    at index end, with loockback days of bptt.
    
    Parameters
    ----------
    data: series, dataframe or array
        the data, containing one row for each time point and one column for each feature
        
    start: int
        starting index in the data
        
    end: int
        index where the whole thing ends, data[end] is **excluded**
        
    lookback: int
        length of the lookback period
        
    Returns
    -------
    X: np.ndarray
        array of shape(n_points, lookback, n_features)
    """
    if isinstance(data, np.ndarray):
        if data.ndim == 1:
            my_data = data.reshape(data.shape[0], 1)
        elif data.ndim == 2:
            my_data = data
    elif isinstance(data, pd.Series):
        my_data = data.values.reshape(data.shape[0], 1)
    elif isinstance(data, pd.DataFrame):
        my_data = data.values
    else:
        raise TypeError("data should be pd.Series, pd.DataFrame or np.ndarray")

    assert lookback < my_data.shape[0]  # lookback sano
    assert start - lookback + 1 >= 0  # lookback sano
    
    n_features = my_data.shape[1]
    n_points = end - start
    
    X = np.zeros((n_points, lookback, n_features), dtype = my_data.dtype)
    
    # range strano per l'indicizzazione numpy
    for i, t in enumerate(range(start + 1, end + 1)):
        X[i, :, :] = my_data[t - lookback:t, :]
        
    return X


# testata, funziona hehehe
def rolling_window_xy(data: Union[np.ndarray, pd.Series, pd.DataFrame],
                     target: Union[np.ndarray, pd.Series],
                     start: int,
                     end: int,
                     lookback: int):
    """
    Create X and y in a single shot.
    """
    
    X = rolling_window(data, start, end, lookback)
    
    if isinstance(target, pd.Series):
        y = target.values[start + 1:end + 1]
    elif isinstance(target, np.ndarray):
        y = target[start + 1:end + 1]
    else:
        raise TypeError("target should be a pandas Series or numpy array")
        
    return X, y

### 2.2 Divisione dataset

Ora che abbiamo le due funzioni, dividiamo il dataset in training e testing, utilizzando le date di prima.

In [ ]:
test_start_1 = returns_all['min_vol'].index.get_loc(split_dates['subprime-crisis-halfway'])
test_end_1 = returns_all['min_vol'].index.get_loc(split_dates['eu-debt-halfway'])



In [ ]:
test_start_1